### Timeseries request Example

Import necessary libraries (Uncomment logging if you want to enable it)

In [11]:
#import logging

#logger = logging.getLogger('')
#logger.setLevel(logging.DEBUG)

import os
import json

from covjsonkit.api import Covjsonkit
from polytope_mars.api import PolytopeMars
from conflator import Conflator
from polytope_mars.config import PolytopeMarsConfig 

Load options needed to pull operational data

In [2]:
options = {
            "axis_config": [
                {
                    "axis_name": "date",
                    "transformations": [{"name": "merge", "other_axis": "time", "linkers": ["T", "00"]}],
                },
                {
                    "axis_name": "values",
                    "transformations": [
                        {
                            "name": "mapper",
                            "type": "octahedral",
                            "resolution": 1280,
                            "axes": ["latitude", "longitude"],
                        }
                    ],
                },
                {"axis_name": "latitude", "transformations": [{"name": "reverse", "is_reverse": True}]},
                {"axis_name": "longitude", "transformations": [{"name": "cyclic", "range": [0, 360]}]},
                {"axis_name": "step", "transformations": [{"name": "type_change", "type": "int"}]},
                {"axis_name": "number", "transformations": [{"name": "type_change", "type": "int"}]},
            ],
            "compressed_axes_config": [
                "longitude",
                "latitude",
                "levtype",
                "step",
                "date",
                "domain",
                "expver",
                "param",
                "class",
                "stream",
                "type",
            ],
            "pre_path": {"class": "od", "expver": "0001", "levtype": "sfc", "stream": "oper", "type": "fc"},
        }

Make polytope-mars request for three parameters at a given point

In [3]:
# time series test
request = {
    "class": "od",
    "stream" : "oper",
    "type" : "fc",
    "date" : "20240915",
    "time" : "0000",
    "levtype" : "sfc",
    "expver" : "0001", 
    "domain" : "g",
    "param" : "167/168/169",
    "step" : "0/to/10",
    "feature" : {
        "type" : "timeseries",
        "points": [[38, -9.5]],
        "axis": "step",
    },
}

result = PolytopeMars().extract(request)

Gribjump: Union request is retrieve,class=od,date=20240915,time=0000,domain=g,expver=0001,levtype=sfc,param=169/168/167,step=0/1/2/3/4/5/6/7/8/9/10,stream=oper,type=fc
Gribjump Engine: Flattened requests and constructed union request: 0.086907 second elapsed, 0.085705 second cpu
Gribjump Engine: Called fdb.list and constructed file map: 0.090474 second elapsed, 0.077734 second cpu
Starting 8 threads
Gribjump Engine: Enqueued 6 file tasks: 0.000913 second elapsed, 0.001264 second cpu
Gribjump Progress: 1 of 6 tasks complete
Gribjump Progress: 2 of 6 tasks complete
Gribjump Progress: 3 of 6 tasks complete
Gribjump Progress: 4 of 6 tasks complete
Gribjump Progress: 5 of 6 tasks complete
Gribjump Progress: 6 of 6 tasks complete
Gribjump Engine: All tasks finished: 0.56963 second elapsed, 0.43168 second cpu
Gribjump Engine: Repackaged results: 0.000213 second elapsed, 0.0004 second cpu


Show resulting covjson

In [4]:
result

{'type': 'CoverageCollection',
 'domainType': 'PointSeries',
 'coverages': [{'mars:metadata': {'class': 'od',
    'Forecast date': '2024-09-15T00:00:00Z',
    'domain': 'g',
    'expver': '0001',
    'levtype': 'sfc',
    'stream': 'oper',
    'type': 'fc',
    'number': 0},
   'type': 'Coverage',
   'domain': {'type': 'Domain',
    'axes': {'x': {'values': [37.99648420814]},
     'y': {'values': [350.443548387097]},
     'z': {'values': [0]},
     't': {'values': ['2024-09-15T00:00:00Z',
       '2024-09-15T01:00:00Z',
       '2024-09-15T02:00:00Z',
       '2024-09-15T03:00:00Z',
       '2024-09-15T04:00:00Z',
       '2024-09-15T05:00:00Z',
       '2024-09-15T06:00:00Z',
       '2024-09-15T07:00:00Z',
       '2024-09-15T08:00:00Z',
       '2024-09-15T09:00:00Z',
       '2024-09-15T10:00:00Z']}}},
   'ranges': {'ssrd': {'type': 'NdArray',
     'dataType': 'float',
     'shape': [11],
     'axisNames': ['ssrd'],
     'values': [0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,


Convert and display as xarray

In [5]:
decoder = Covjsonkit().decode(result)
ds = decoder.to_xarray()
ds

<xarray.Dataset> Size: 464B
Dimensions:   (x: 1, y: 1, z: 1, number: 1, datetime: 1, t: 11)
Coordinates:
  * x         (x) float64 8B 38.0
  * y         (y) float64 8B 350.4
  * z         (z) int64 8B 0
  * number    (number) int64 8B 0
  * datetime  (datetime) <U20 80B '2024-09-15T00:00:00Z'
  * t         (t) datetime64[ns] 88B 2024-09-15 ... 2024-09-15T10:00:00
Data variables:
    ssrd      (x, y, z, number, datetime, t) float64 88B 0.0 0.0 ... 3.784e+06
    2d        (x, y, z, number, datetime, t) float64 88B 287.1 287.2 ... 287.5
    2t        (x, y, z, number, datetime, t) float64 88B 292.8 292.6 ... 291.5
Attributes:
    class:          od
    Forecast date:  2024-09-15T00:00:00Z
    domain:         g
    expver:         0001
    levtype:        sfc
    stream:         oper
    type:           fc
    number:         0

Change options to pull ensemble data

In [6]:
options = {
            "axis_config": [
                {
                    "axis_name": "date",
                    "transformations": [{"name": "merge", "other_axis": "time", "linkers": ["T", "00"]}],
                },
                {
                    "axis_name": "values",
                    "transformations": [
                        {
                            "name": "mapper",
                            "type": "octahedral",
                            "resolution": 1280,
                            "axes": ["latitude", "longitude"],
                        }
                    ],
                },
                {"axis_name": "latitude", "transformations": [{"name": "reverse", "is_reverse": True}]},
                {"axis_name": "longitude", "transformations": [{"name": "cyclic", "range": [0, 360]}]},
                {"axis_name": "step", "transformations": [{"name": "type_change", "type": "int"}]},
                {"axis_name": "number", "transformations": [{"name": "type_change", "type": "int"}]},
            ],
            "compressed_axes_config": [
                "longitude",
                "latitude",
                "levtype",
                "step",
                "date",
                "domain",
                "expver",
                "param",
                "class",
                "stream",
                "type",
                "number",
            ],
            "pre_path": {"class": "od", "expver": "0001", "levtype": "sfc", "stream": "enfo", "type": "pf"},
        }

Replace default options in config with emsemble options

In [7]:
conf = Conflator(app_name="polytope_mars", model=PolytopeMarsConfig).load()
cf = conf.model_dump()
cf['options'] = options
cf

{'datacube': {'type': 'gribjump',
  'config': 'config.yaml',
  'uri': 'http://localhost:8001'},
 'options': {'axis_config': [{'axis_name': 'date',
    'transformations': [{'name': 'merge',
      'other_axis': 'time',
      'linkers': ['T', '00']}]},
   {'axis_name': 'values',
    'transformations': [{'name': 'mapper',
      'type': 'octahedral',
      'resolution': 1280,
      'axes': ['latitude', 'longitude']}]},
   {'axis_name': 'latitude',
    'transformations': [{'name': 'reverse', 'is_reverse': True}]},
   {'axis_name': 'longitude',
    'transformations': [{'name': 'cyclic', 'range': [0, 360]}]},
   {'axis_name': 'step',
    'transformations': [{'name': 'type_change', 'type': 'int'}]},
   {'axis_name': 'number',
    'transformations': [{'name': 'type_change', 'type': 'int'}]}],
  'compressed_axes_config': ['longitude',
   'latitude',
   'levtype',
   'step',
   'date',
   'domain',
   'expver',
   'param',
   'class',
   'stream',
   'type',
   'number'],
  'pre_path': {'class': '

Request ensemble data

In [8]:
# time series test
request = {
    "class": "od",
    "stream" : "enfo",
    "type" : "pf",
    "date" : "20240915",
    "time" : "0000",
    "levtype" : "sfc",
    "expver" : "0001", 
    "domain" : "g",
    "param" : "167/169",
    "number" : "1/to/10",
    "step": "0/to/50",
    "feature" : {
        "type" : "timeseries",
        "points": [[38, -9.5]],
        "axis": "step",
    },
}

result = PolytopeMars(cf).extract(request)

Gribjump: Union request is retrieve,class=od,date=20240915,time=0000,domain=g,expver=0001,levtype=sfc,number=1/2/3/4/5/6/7/8/9/10,param=167/169,step=0/1/2/3/4/5/6/7/8/9/10/11/12/13/14/15/16/17/18/19/20/21/22/23/24/25/26/27/28/29/30/31/32/33/34/35/36/37/38/39/40/41/42/43/44/45/46/47/48/49/50,stream=enfo,type=pf
Gribjump Engine: Flattened requests and constructed union request: 2 seconds elapsed, 2 seconds cpu
Gribjump Engine: Called fdb.list and constructed file map: 0.738831 second elapsed, 0.668127 second cpu
Gribjump Engine: Enqueued 30 file tasks: 0.000144 second elapsed, 0.000468 second cpu
Gribjump Progress: 1 of 30 tasks complete
Gribjump Progress: 4 of 30 tasks complete
Gribjump Progress: 7 of 30 tasks complete
Gribjump Progress: 10 of 30 tasks complete
Gribjump Progress: 13 of 30 tasks complete
Gribjump Progress: 16 of 30 tasks complete
Gribjump Progress: 19 of 30 tasks complete
Gribjump Progress: 22 of 30 tasks complete
Gribjump Progress: 25 of 30 tasks complete
Gribjump Progr

In [9]:
result

{'type': 'CoverageCollection',
 'domainType': 'PointSeries',
 'coverages': [{'mars:metadata': {'class': 'od',
    'Forecast date': '2024-09-15T00:00:00Z',
    'domain': 'g',
    'expver': '0001',
    'levtype': 'sfc',
    'number': 1,
    'stream': 'enfo',
    'type': 'pf'},
   'type': 'Coverage',
   'domain': {'type': 'Domain',
    'axes': {'x': {'values': [37.99648420814]},
     'y': {'values': [350.443548387097]},
     'z': {'values': [0]},
     't': {'values': ['2024-09-15T00:00:00Z',
       '2024-09-15T01:00:00Z',
       '2024-09-15T02:00:00Z',
       '2024-09-15T03:00:00Z',
       '2024-09-15T04:00:00Z',
       '2024-09-15T05:00:00Z',
       '2024-09-15T06:00:00Z',
       '2024-09-15T07:00:00Z',
       '2024-09-15T08:00:00Z',
       '2024-09-15T09:00:00Z',
       '2024-09-15T10:00:00Z',
       '2024-09-15T11:00:00Z',
       '2024-09-15T12:00:00Z',
       '2024-09-15T13:00:00Z',
       '2024-09-15T14:00:00Z',
       '2024-09-15T15:00:00Z',
       '2024-09-15T16:00:00Z',
       '20

In [10]:
decoder = Covjsonkit().decode(result)
ds = decoder.to_xarray()
ds

<xarray.Dataset> Size: 9kB
Dimensions:   (x: 1, y: 1, z: 1, number: 10, datetime: 1, t: 51)
Coordinates:
  * x         (x) float64 8B 38.0
  * y         (y) float64 8B 350.4
  * z         (z) int64 8B 0
  * number    (number) int64 80B 1 2 3 4 5 6 7 8 9 10
  * datetime  (datetime) <U20 80B '2024-09-15T00:00:00Z'
  * t         (t) datetime64[ns] 408B 2024-09-15 ... 2024-09-17T02:00:00
Data variables:
    2t        (x, y, z, number, datetime, t) float64 4kB 292.8 292.5 ... 292.3
    ssrd      (x, y, z, number, datetime, t) float64 4kB 0.0 0.0 ... 4.22e+07
Attributes:
    class:          od
    Forecast date:  2024-09-15T00:00:00Z
    domain:         g
    expver:         0001
    levtype:        sfc
    number:         1
    stream:         enfo
    type:           pf

Save output to covjson for use in earrthkit

In [10]:
with open('test_timeseries.covjson', 'w') as fp:
    json.dump(result, fp)

Plot meteogram

In [11]:
import ipywidgets as widgets
import earthkit.plots
import earthkit.data

TIME_FREQUENCY = "1H"

def f():
    data = earthkit.data.from_source("file", "test_timeseries.covjson")
    chart = earthkit.plots.Chart()
    chart.box(data, time_frequency=TIME_FREQUENCY)
    chart.line(data, time_frequency=TIME_FREQUENCY, aggregation="mean", line_color="purple")
    chart.show()

out = widgets.interactive_output(f, {})
display(out)

Output()

In [ ]:
earthkit.data("polytope", "operational", request)